# Visualização de Dados Covid 19

## Alunos
Alexandre Brito Gomes - 11857323

Diógenes Silva Pedro - 11883476

Gabriel Freitas Ximenes Vasconcelos - 11819084

Nesta visualização, analisaremos o conjunto de dados "Coronavirus - Brazil", disponível no Kaggle em https://www.kaggle.com/datasets/unanimad/corona-virus-brazil?select=brazil_covid19_cities.csv . Nele, é possível obter informações sobre o número de casos de Coronavirus no Brasil de maneira temporal: apresentando o número de casos, de maneira cumulativa, desde 30 de janeiro de 2020, quando o primeiro caso suspeito foi encontrado no país. 

Esse conjunto de dados é composto por 6 arquivos csv, contendo dados como latitude e longitude das cidades, data da confirmação de primeiro caso, número de casos confirmados, número de mortes, recuperados, suspeitos, entre diversos outros.

Com isso, é possível fazer análises como verificação de quais localidades tiveram maiores números de casos em pouco tempo, como foi a evolução dos casos ao longo do tempo 

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import json
from urllib.request import urlopen
import math
from collections import OrderedDict

# Importação dos datasets base

In [ ]:
data_path = 'covid-data/'

In [ ]:
sigla_estado = json.load(open("sigla_estado.json", encoding="utf-8"))

In [ ]:
def get_sigla(row):
    return sigla_estado[row['state']]

In [ ]:
df_population = pd.read_csv(data_path + 'brazil_population_2019.csv')
df_population["state_sigla"] = df_population.apply(get_sigla, axis=1)
df_population.rename({'state': 'full_state'}, inplace=True, axis=1)
df_population

In [ ]:
df_cities = pd.read_csv(data_path + "brazil_covid19_cities.csv")
df_cities

In [ ]:
states = df_population.drop_duplicates(['full_state'])
area_dict = dict(zip(states.state_code, states.state_sigla))
print(area_dict)
def get_sigla(row):
    return area_dict[row['state_code']]

In [ ]:
df_coordinates = pd.read_csv(data_path + "brazil_cities_coordinates.csv")
df_coordinates['state_sigla'] = df_coordinates.apply(get_sigla, axis=1)
df_coordinates

# Criação do df_city

In [ ]:
df_city = df_cities.merge(df_population[['city', 'population', 'region', 'full_state', 'state_sigla']], how='inner', left_on = ['name', 'state'], right_on=['city', 'state_sigla'])
df_city = df_city.merge(df_coordinates[['city_name', 'city_code', 'lat', 'long', 'state_code', 'state_sigla']], how='inner', left_on=['name', 'state'], right_on=['city_name', 'state_sigla'])
df_city = df_city.drop(["code", "name", "city_name", "state_sigla_x", "state_sigla_y"], axis='columns')
df_city

In [ ]:
df_city.sort_values(['city', 'state', 'date'], inplace=True)
df_city.reset_index(inplace=True, drop=True)

## Resolver o problema da acumulação

As colunas `death` e `mortes` são colunas que idealmente deveriam ser acumuladas. No entanto, ao analisar o dataset podemos ver que há casos em que valores diminuem.
O dataset possui 2770 municípios e apenas 266 não possuem esse problema, por isso, devemos dar a devida atenção a esse problema nos dados.

Para amenizar isso, criamos a função `fix_accumulation` que funciona da seguinte maneira:

1. Defino como `start` a posição cuja a posição seguinte é um valor menor
2. Percorro o array até achar uma posição cujo valor é igual ou superior ao valor do `start`. Essa posição é a `end`
3. Vejo a quantidade de elementos entre `start` e `end` e faço a seguinte equação que representa o coeficiente angular da reta:
   $$
   \Delta y = \frac{\text{arr[end]} - \text{arr[start]}}{\text{end}-\text{start}}
   $$
4. Incremento $\Delta y$ a partir de cada valor, começando de `start` até chegar ao `end`
5. Caso um `start` comece e o array termine antes de chegar ao `end`, os valores seguintes ao `start` são substituídos por ele

In [ ]:
def fix_accumulation(original_arr):
    arr = original_arr.copy()
    prev = arr[0]
    start = 0
    end = 0
    interval = False
    for i, el in enumerate(arr):
        if el < prev and interval == False:
            start = i-1
            interval = True
            
        if el >= arr[start] and interval == True:
            end = i
            interval=False
            delta = (arr[end]-arr[start])/(end-start)
            for j in range(start+1, end):
                arr[j] = int(math.floor(arr[j-1] + delta))
                
        prev = arr[i]
        
    if interval == True:
        for i in range(start+1, len(arr)):
            arr[i] = arr[start]
            
    return arr

In [ ]:
def solve_column(coluna):
    problemas_city = list(df_city.loc[df_city[coluna + ' per day'] < 0].drop_duplicates(subset=['city', 'state'], inplace=False)["city"])
    problemas_state = list(df_city.loc[df_city[coluna + ' per day'] < 0].drop_duplicates(subset=['city', 'state'], inplace=False)["state"])
    print(f"Cidades com o problema na coluna {coluna}: {len(problemas_city)}")

    indexes = []
    for i in range(len(problemas_city)):
        citystate_index = df_city.loc[(df_city['city'] == problemas_city[i]) & (df_city['state'] == problemas_state[i])].index
        indexes.append(citystate_index)

    lists_of_cases = [df_city.iloc[city][coluna].to_list() for city in indexes]
    fixed_cases = [fix_accumulation(city) for city in lists_of_cases]

    for i in range(len(fixed_cases)):
        city_indexes = indexes[i]
        fixed_city = fixed_cases[i]
        df_city.loc[city_indexes, coluna]=fixed_city

In [ ]:
# Criamos as colunas `cases per day` e `mortes per day` para sabermos onde isso dá problema

cum_columns = ['cases', 'deaths']
df_city = df_city.merge(
    df_city.groupby(['city', 'state'])[cum_columns].diff(),
    left_index=True, right_index=True, suffixes=['', ' per day']
).fillna({'{} per day'.format(cum_column): df_city[cum_column] for cum_column in cum_columns})

df_city.head()

In [ ]:
solve_column("cases")
solve_column("deaths")

In [ ]:
# dropo as colunas antigas incorretas
df_city = df_city.drop(columns=['cases per day', 'deaths per day'])

# recrio as colunas só que com a agregação correta
cum_columns = ['cases', 'deaths']
df_city = df_city.merge(
    df_city.groupby(['city', 'state'])[cum_columns].diff(),
    left_index=True, right_index=True, suffixes=['', ' per day']
).fillna({'{} per day'.format(cum_column): df_city[cum_column] for cum_column in cum_columns})

df_city.head()

In [ ]:
# Verifico se não há nenhuma coluna com problema de acumulação após o processamento
quant_cases = df_city.loc[df_city['cases per day'] < 0].drop_duplicates(subset=['city', 'state']).shape[0]
quant_deaths = df_city.loc[df_city['deaths per day'] < 0].drop_duplicates(subset=['city', 'state']).shape[0]
print(f"Cidades com o problema na coluna cases após correção: {quant_cases}")
print(f"Cidades com o problema na coluna deaths após correção: {quant_deaths}")

In [ ]:
# Obtenho uma normalização por meio do tamanho da população da cidade
df_city["cases/population"] = df_city["cases"]/df_city["population"]
df_city["deaths/population"] = df_city["deaths"]/df_city["population"]
df_city["cases per day/population"] = df_city["cases per day"]/df_city["population"]
df_city["deaths per day/population"] = df_city["deaths per day"]/df_city["population"]

df_city.rename(columns={'long': 'lon'}, inplace=True)

df_city.head()

In [ ]:
df_city.to_csv('etl_data/city.csv', index=True)

# Criação do df_state

In [ ]:
# obtenho o número de casos e mortes por dia por estado
df_state = df_city.groupby(['state', 'date'])[['cases per day', 'deaths per day']].sum()
df_state.reset_index(inplace=True)

In [ ]:
# adiciono a população do estado, somando as populações dos municípios do estado
state_population = df_city.drop_duplicates(subset=['city', 'state']).groupby('state')['population'].sum()
population_dict = state_population.to_dict()

def add_population(row):
    return population_dict[row['state']]

df_state['population'] = df_state.apply(add_population, axis=1)
df_state

In [ ]:
# calculo uma média da latitude e longitude dos municípios do estado
state_avg_coord = df_city.drop_duplicates(subset=['city', 'state']).groupby('state')[['lat', 'lon']].mean()

# adiciono a lat e long média, além de adicionar a região do estado
df_state = df_state.merge(state_avg_coord, on='state').merge(df_city[['state', 'region']].drop_duplicates(subset=['state']), on='state')

# calculo e adiciono o número de casos e mortes acumulados do estado
df_state['cases'] = df_state.groupby('state')['cases per day'].cumsum(axis=0)
df_state['deaths'] = df_state.groupby('state')['deaths per day'].cumsum(axis=0)

# Obtenho uma normalização por meio do tamanho da população do estado
df_state["cases/population"] = df_state["cases"]/df_state["population"]
df_state["deaths/population"] = df_state["deaths"]/df_state["population"]
df_state["cases per day/population"] = df_state["cases per day"]/df_state["population"]
df_state["deaths per day/population"] = df_state["deaths per day"]/df_state["population"]
df_state

In [ ]:
df_state.to_csv('etl_data/state.csv', index=True)

# Criação do df_region

In [ ]:
# obtenho o número de casos e mortes por dia por estado
df_region = df_state.groupby(['region', 'date'])[['cases per day', 'deaths per day']].sum()
df_region.reset_index(inplace=True)

In [ ]:
# adiciono a população do estado, somando as populações dos municípios do estado
region_population = df_state.drop_duplicates(subset=['state']).groupby('region')['population'].sum()
population_dict = region_population.to_dict()

def add_population(row):
    return population_dict[row['region']]

df_region['population'] = df_region.apply(add_population, axis=1)
df_region

In [ ]:
# calculo uma média da latitude e longitude dos municípios do estado
region_avg_coord = df_state.drop_duplicates(subset=['state']).groupby('region')[['lat', 'lon']].mean()

# adiciono a lat e long média, além de adicionar a região do estado
df_region = df_region.merge(region_avg_coord, on='region')

# calculo e adiciono o número de casos e mortes acumulados do estado
df_region['cases'] = df_region.groupby('region')['cases per day'].cumsum(axis=0)
df_region['deaths'] = df_region.groupby('region')['deaths per day'].cumsum(axis=0)

# Obtenho uma normalização por meio do tamanho da população do estado
df_region["cases/population"] = df_region["cases"]/df_region["population"]
df_region["deaths/population"] = df_region["deaths"]/df_region["population"]
df_region["cases per day/population"] = df_region["cases per day"]/df_region["population"]
df_region["deaths per day/population"] = df_region["deaths per day"]/df_region["population"]
df_region

In [ ]:
df_region.to_csv('etl_data/region.csv', index=True)

# Carregar CSVs prontos

In [ ]:
df_city = pd.read_csv("etl_data/city.csv", index_col=0)
df_state = pd.read_csv("etl_data/state.csv", index_col=0)
df_region = pd.read_csv("etl_data/region.csv", index_col=0)

# Criação do gráfico

In [ ]:
df_cities_coordinates = pd.read_csv(data_path + "brazil_cities_coordinates.csv")
df_cities_coordinates.head()

In [ ]:
df_region.drop_duplicates('region')[["lat", "lon"]].mean().to_dict()

In [ ]:
brazil_code = 100
with urlopen(f'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-{brazil_code}-mun.json') as response:
    brazil_json = json.loads(response.read())

In [ ]:
df_city.loc[(df_city['city'] == 'São Carlos') & (df_city['state'] == 'SP')]

In [ ]:
df_city.iloc[1024081]

In [ ]:
df_city.loc[(df_city['city'] == 'São Carlos')]

In [ ]:
fig = px.choropleth_mapbox(df_city.loc[df_city['date'] == '2021-03-29'], geojson=brazil_json, featureidkey = 'properties.id', locations='city_code', color='cases/population',
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=3, center = df_region.drop_duplicates('region')[["lat", "lon"]].mean().to_dict(),
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()